<div style='background-image: url("../share/Aerial_view_LLNL.jpg") ; padding: 0px ; background-size: cover ; border-radius: 15px ; height: 250px; background-position: 0% 80%'>
    <div style="float: right ; margin: 50px ; padding: 20px ; background: rgba(255 , 255 , 255 , 0.8) ; width: 50% ; height: 150px">
        <div style="position: relative ; top: 50% ; transform: translatey(-50%)">
            <div style="font-size: xx-large ; font-weight: 900 ; color: rgba(0 , 0 , 0 , 0.9) ; line-height: 100%">2014 South Napa Earthquake</div>
            <div style="font-size: large ; padding-top: 20px ; color: rgba(0 , 0 , 0 , 0.7)">Large Scale Downloading Exercise</div>
        </div>
    </div>
</div>

### **2017 CIG-LLNL Computational Seismology Workshop**


##### Authors:
* Artie Rogers
* Lion Krischer ([@krischer](https://github.com/krischer))
---

This notebooks aims to teach you how to use ObsPy to download a larger data-set that you can then use for example for seismic inversions, sources studies, or whatever else suits your fancy. In this example we will download waveform and station data in certain geographical region for the South Napa earthquake.

In [ ]:
%matplotlib inline
from __future__ import print_function
import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 12, 8

### Event Data

The first step again is do find the event.

In [ ]:
# You shopuld be able to do this by yourself by now. Best use the 
# USGS FDSN web service but others might work as well.

...

### Waveform and Station Data

You could once again write loops and manually download waveforms and corresponding station files like seen in the previous exercises. This quickly gets tedious and error prone, especially if one wants to incorporate data from multiple data centers.

To this end IRIS developed the so-called [Federator](https://ds.iris.edu/ds/newsletter/vol17/no2/444/introducing-the-iris-federator-time-series-data-search-across-global-data-centers/) which enables you to figure out which data centers hosts what data. The counter-part for the European ORFEUS consortium is the [EIDAWS Routing service](http://www.orfeus-eu.org/data/eida/webservices/routing/). Unfortunately we did not manage to get support for these into ObsPy in time for this workshop. But it will happen very soon.

Chad will talk more about the federator in the afternoon.


#### Mass Downloader

ObsPy has a submodule to help with this, the mass downloader. It attempts to offer an API for how seismologists would like to download data. It works in three steps:

1. Define Geographical Domain
2. Define Other Restrictions
3. Launch Download

After this it fully automatically downloads across all data centers it knows of. Many seismologist want to solve a problem on Earth an don’t care that much where the data actually comes from. Additionally it has a couple of other nice features:

* Automatic Quality Control
* Define Other Restrictions
* Relaunch Downloads
* One always need waveform data and station meta information
* Acquire a “nice and complete” dataset
* Usable as a library


#### Exercise


##### Part A

Use the mass downloader to download data for a small (there are a lot of stations in this region) geographical region (a circular region with a radius of 1 degree is an example), including the station information, with the mass downloader. Download from 2 minutes before the event to 10 minutes after it and download only `LHZ` channels.

**The documentation is your friend. So is just searching the internet.**

In [ ]:
import obspy
from obspy.clients.fdsn.mass_downloader import CircularDomain, \
    Restrictions, MassDownloader


# First, define a domain.
domain = CircularDomain(...)

# Second, define some additional restrictions.
restrictions = Restrictions(...)

# If you leave the providers empty it will loop through
# all data centers it knows.
# It will prefer data from the first providers.
mdl = MassDownloader(providers=["SCEDC", "NCEDC", "IRIS"])

# Finally launch it.
mdl.download(domain, restrictions,
             # The files will be stored in these two folders.
             mseed_storage="waveforms",
             stationxml_storage="stations")

In [ ]:
!ls -l waveforms
!ls -l stations

##### Bonus Exercise

Plot the stations and the event in a single map. Note that `obspy.read_inventory("*.xml")` does not work in ObsPy 1.0.3 (it does work in the master though...). Use the `glob` module of Python's stdlib.

In [ ]:
...

##### Part B

Process the data in the following way and store each trace in a new folder as a SAC file:

1. Instrument correction to displacement
2. Detrend and taper
3. Apply a bandpass filter from 100 to 20 seconds.
4. Sample all at exactly the same points in time at a sampling rate of 0.5 Hz.

There is a bug in ObsPy 1.0.3 with new numpy versions
(fixed in the master once again...) that will cause
an exception when running the Lanczos interpolation
after any filter. To fix it just do the following right
before the Lanczos interpolation:

```python
import numpy as np
for tr in st:
    tr.data = np.require(tr.data,
                         requirements=["C_CONTIGUOUS"])
```

In [ ]:
...

In [ ]:
!ls SAC_files/